In [ ]:
import pandas as pd
import plotly.express as px

#### Импорт таблиц

In [ ]:
ab_group = pd.read_csv('ABgroup.csv')

In [ ]:
cheaters = pd.read_csv('Cheaters.csv')
cash = pd.read_csv('Cash.csv')
money = pd.read_csv('Money.csv')

#### Первичный осмотр данных

In [ ]:
ab_group.info()

In [ ]:
cheaters.info()

In [ ]:
cash.info()

In [ ]:
money.info()

#### Проверка и удаление дубликатов в таблицах

In [ ]:
ab_group.duplicated().sum()

In [ ]:
ab_group = ab_group.drop_duplicates(subset=['user_id'])

In [ ]:
cheaters.duplicated().sum()

In [ ]:
cheaters = cheaters.drop_duplicates(subset=['user_id'])

#### Суммирование платжей/кэша по user id

In [ ]:
money_groupped = money.groupby('user_id', as_index=False).money.sum()
cash_groupped = cash.groupby('user_id', as_index=False).cash.sum()

#### Join таблиц

In [ ]:
id_cheaters = ab_group.merge(cheaters, on='user_id', how='left').fillna(0)

In [ ]:
id_cheaters = id_cheaters.merge(cash_groupped, on='user_id', how='left').fillna(0)
id_cheaters.head(50)

In [ ]:
id_cheaters = id_cheaters.merge(money_groupped, on='user_id', how='left').fillna(0)
id_cheaters.head(50)

#### Найдем читеров, которые не были зафиксированы

In [ ]:
# читеры
real_cheaters = id_cheaters[id_cheaters.cheaters==1]

In [ ]:
real_cheaters.describe()

In [ ]:
# остальные игроки
rest = id_cheaters[id_cheaters.cheaters==0]

In [ ]:
rest.describe()

In [ ]:
px.histogram(rest, x='cash')

In [ ]:
px.box(rest, y='cash')

In [ ]:
rest.cash.quantile(0.999)

In [ ]:
# Принцип обнаружения читера - неплательщик, который тратил внутриигровую валюту в существенных количествах 

#### Удаляем читеров

In [ ]:
# Удаляем известных читеров
no_cheaters = id_cheaters[id_cheaters.cheaters!=1]

In [ ]:
# Читеры по принципу:
no_cheaters[(no_cheaters.money==0)&(no_cheaters.cash>20000)].user_id.nunique()

In [ ]:
# кол-во юзеров в таблице до удаления обнаруженных читеров 
no_cheaters.user_id.nunique()

In [ ]:
# Удаляем читеров по принципу
no_cheaters = no_cheaters[~((no_cheaters.money==0)&(no_cheaters.cash>20000))]

In [ ]:
# кол-во юзеров в таблице после удаления обнаруженных читеров 
no_cheaters.user_id.nunique()

#### Итоговая таблица

In [ ]:
no_cheaters.head()

In [ ]:
no_cheaters.info()

In [ ]:
no_cheaters.to_csv('no_cheaters.csv', index=False)